original: https://doordash-dev.cloud.databricks.com/editor/notebooks/889012255470022?o=1953101234786001

In [0]:
!pip install openai
!pip install -U portkey-ai
!pip install python-dotenv
!pip install -qU langchain-google-vertexai

!pip install --upgrade typing_extensions
!pip install langchain faiss-cpu google-cloud-aiplatform
!pip install -qU langchain-google-vertexai langchain-openai langchain-chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.4 MB/s eta 0:00:00
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Not uninstalling h11 at /databricks/python3/lib/python3.12/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-4e5984bc-af46-40d0-9508-80ee3a10d007
    Can't uninstall 'h11'. No files were found to uninstall.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.4 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency co

In [0]:
dbutils.library.restartPython()

In [0]:
import os

os.environ["PORTKEY_API_KEY"] = dbutils.secrets.get(scope="songxu-scope", key="PORTKEY_API_KEY")
# os.environ["OPENAI_API_KEY"] = dbutils.secrets.get(scope="heathersong-scope", key="virtual-api-key-openai-oe")
# os.environ["VIRTUAL_API_KEY"] = dbutils.secrets.get(scope="heathersong-scope", key="virtual-api-key-openai-oe")
os.environ["VERTEX_AI_VIRTUAL_API_KEY"] = dbutils.secrets.get(scope="songxu-scope", key="VERTEXAI_VIRTUAL_KEY")
os.environ["SNOWFLAKE_USER"] = dbutils.secrets.get(scope="songxu-scope", key="snowflake-user")
os.environ["SNOWFLAKE_PASSWORD"] = dbutils.secrets.get(scope="songxu-scope", key="snowflake-password")
os.environ["SNOWFLAKE_WAREHOUSE"] = 'ADHOC'
os.environ["SNOWFLAKE_ROLE"] ='SONGXU'
os.environ["PORTKEY_GATEWAY_URL"] = "http://cybertron-service-gateway.svc.ddnw.net:8080/v1"

PORTKEY_API_KEY = dbutils.secrets.get(scope="songxu-scope", key="PORTKEY_API_KEY")
# OPENAI_API_KEY = dbutils.secrets.get(scope="heathersong-scope", key="virtual-api-key-openai-oe")
VERTEX_AI_VIRTUAL_API_KEY = dbutils.secrets.get(scope="songxu-scope", key="VERTEXAI_VIRTUAL_KEY")
PORTKEY_GATEWAY_URL = "http://cybertron-service-gateway.svc.ddnw.net:8080/v1"

import sys
from pathlib import Path

p = Path.cwd()
while p != p.parent and not (p / 'llm_as_a_judge' / '__init__.py').exists():
    p = p.parent
sys.path.insert(0, str(p))

from llm_as_a_judge.utils.structured_query_store_evaluator import *
from llm_as_a_judge.utils.base_evaluator import *
import pandas as pd
from dataclasses import dataclass

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-1528494400672131>, line 3
      1 import os
----> 3 os.environ["PORTKEY_API_KEY"] = dbutils.secrets.get(scope="songxu-scope", key="PORTKEY_API_KEY")
      4 # os.environ["OPENAI_API_KEY"] = dbutils.secrets.get(scope="heathersong-scope", key="virtual-api-key-openai-oe")
      5 # os.environ["VIRTUAL_API_KEY"] = dbutils.secrets.get(scope="heathersong-scope", key="virtual-api-key-openai-oe")
      6 os.environ["VERTEX_AI_VIRTUAL_API_KEY"] = dbutils.secrets.get(scope="songxu-scope", key="VERTEXAI_VIRTUAL_KEY")

File /databricks/python_shell/lib/dbruntime/dbutils.py:359, in DBUtils.SecretsHandler.get(self, scope, key)
    357 def get(self, scope, key):
    358     return self.entry_point.getDbutils().preview().secret(
--> 359     ).get(  # type: ignore[attr-defined]
    360         scope, key)

File /databricks/spark/python/lib

In [0]:
import pandas as pd

### Step 00 - before pivoting

In [0]:
dbutils.fs.cp(
  "dbfs:/songxu/llm/output_structured_query_dsl_v2_sot_max100_20251027_2.csv",
  "file:/tmp/output_structured_query_dsl_v2_sot_max100_20251027_2.csv"
)

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-5588582825168644>, line 1
----> 1 dbutils.fs.cp(
      2   "dbfs:/songxu/llm/output_structured_query_dsl_v2_sot_max100_20251027_2.csv",
      3   "file:/tmp/output_structured_query_dsl_v2_sot_max100_20251027_2.csv"
      4 )

File /databricks/python_shell/lib/dbruntime/remotefshandler/RemoteFsHandler.py:52, in prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
     49 class ExecutionError(Exception):
     50     pass
---> 52 raise ExecutionError(str(e)) from None

ExecutionError: (org.apache.spark.SparkSecurityException) [INSUFFICIENT_PERMISSIONS] Insufficient privileges:
User does not have permission SELECT on any file. SQLSTATE: 42501

JVM stacktrace:
org.apache.spark.SparkSecurityException
	at com.databricks.sql.acl.Unauthorized.throwInsufficientPermissionsError(PermissionChecker.scala:360)
	

In [0]:
df_raw = pd.read_csv("/dbfs/songxu/llm/output_structured_query_dsl_v2_sot_max100_20251027_2.csv")
data_tag = "max100_20251027_2"
print(df_raw.shape)
df_raw.head(2)

In [0]:
# Extract original_query
df_raw["original_query"] = df_raw["argo_dsl"].apply(lambda x: json.loads(x)["original_query"])
df_raw["query"] = df_raw["original_query"]
# print(df_raw["original_query"])

In [0]:
df_raw = df_raw[df_raw['store_ids']!='[]'][['query', 'store_ids']].drop_duplicates()
print(df_raw.shape)
df_raw.head(2)

In [0]:
import ast

df_raw["store_id"] = df_raw["store_ids"].apply(ast.literal_eval)
df_raw["daypart"] = 'dinner'
df_raw["weekday"] = 'wed'

df_pivot = df_raw[["query", "store_id", "daypart", "weekday"]].explode("store_id")
df_pivot["rank_position"] = df_pivot.groupby("query").cumcount() + 1

# df_pivot = df_pivot.rename(columns={"store_ids": "store_id"})
print(df_pivot.shape)
df_pivot.head(2)

# step 0 - load raw input <query, store_id, daypart> data

In [0]:

# df_all_dedup_df = pd.read_csv("/dbfs/songxu/llm/wrp_20250917_food_finalized_pivoted.csv", sep='\t').rename(columns={"query_text":'query','day_part':'daypart'})
# df = pd.read_csv('/dbfs/songxu/llm/wrp_20250924_food_finalized_pivoted.csv', sep='\t')

df = df_pivot

# df['weekday']= 'wed'
print(df.columns)
df_orig = df
df_all_dedup_df = df[['query','store_id','daypart','weekday']].drop_duplicates()
df_all_dedup_df.display()
# df_all_dedup.display()

# step 1 - set up evaluator

In [0]:
evaluator = StructuredQueryStoreEvaluator(
    user_prompt_column_names_mapping={
      "search_query": "query",
      "most_relevant_20_items_in_the_store":"most_relevant_20_items_in_the_store_core_category",
      "store_summary": "store_summary",
      "store_price_dollar_sign": "store_price_range_sign",
      "store_rating": "store_average_rating",
      "store_and_consumer_distance_miles": "store_consumer_distance_in_miles", 
      "store_eta_minute": "store_eta",
      "store_address":'store_address', 
    },
    structured_llm_call_output_class=BinaryQueryStoreResponse,
    model="gemini-2.5-pro",  # or any model routed via Portkey
    temperature=0.0,
    max_tokens=1024,
    timeout=60,
    prefix="gemini-2.5-pro",
    data_save_path = f'/dbfs/songxu/llm/scrape_{data_tag}' # TODO: will it break anything? 
)

# step 2 prepare data for arize (or running for offline eval)

In [0]:
df_test = df#.iloc[0:10]
df_prepared = evaluator._prepare_data(df_test)
# df_prepared.display()

In [0]:
write_data_spark(df_prepared, f'songxu.structured_query_data_{data_tag}')
df_prepared.write.mode('overwrite').parquet(f'/dbfs/songxu/llm/structured_query_data_{data_tag}')

# step 3 - Run eval

In [0]:
# if evaluator.data_preprocess_save_path:
#   df_prepared_pd = spark.read.parquet(evaluator.data_preprocess_save_path).toPandas()

df_prepared_pd = spark.read.parquet(f'/dbfs/songxu/llm/structured_query_data_{data_tag}').toPandas()

df_eval = evaluator.run_parallel_eval(
  df = df_prepared_pd,
  max_workers= 256
  ) 

# df_eval.display()
# df_eval_post.to_csv('/dbfs/mnt/doordash-datalake/test_tmp/ml/datasets/hs/vox/structured_query_eval_res_post_process.csv', index=False)

In [0]:
df_eval.to_csv(f'/dbfs/songxu/llm/structured_query_eval_res_{data_tag}.csv', index=False, escapechar='\\')

# step 4 run post processing

In [0]:
df_eval_post = evaluator._postprocess_data(df_eval, df_orig)
# df_eval_post.display()

In [0]:
# for max=100
df_metric_dct = evaluator._compute_metrics(df = df_eval_post, k = 5,rank_col_name = 'rank_position', id_cols = ['query','daypart'])
df_metric_dct.keys()

In [0]:
# for max=75
df_metric_dct = evaluator._compute_metrics(df = df_eval_post, k = 5,rank_col_name = 'rank_position', id_cols = ['query','daypart'])
df_metric_dct.keys()

In [0]:
# for max=50
df_metric_dct = evaluator._compute_metrics(df = df_eval_post, k = 5,rank_col_name = 'rank_position', id_cols = ['query','daypart'])
df_metric_dct.keys()

In [0]:
# for max=25
df_metric_dct = evaluator._compute_metrics(df = df_eval_post, k = 5,rank_col_name = 'rank_position', id_cols = ['query','daypart'])
df_metric_dct.keys()

In [0]:
def get_precision_at_k(_df, k):
  __df =  _df[_df['rank_position'] <= k]
  return __df[__df['gemini-2.5-pro.label']=='relevant'].shape[0] / __df.shape[0]

get_precision_at_k(df_eval_post, 100)

In [0]:
k_s = [1, 5, 10, 20, 50, 100]

precision_s = []
for k in k_s:
  precision_s.append(get_precision_at_k(df_eval_post, k))
precision_s

In [0]:
import matplotlib.pyplot as plt
plt.plot(k_s, precision_s)
plt.ylim([0,1])
plt.xlabel('k')
plt.ylabel('precision@k')

# run metric computation

In [0]:
# for max=75
df_metric_dct = evaluator._compute_metrics(df = df_eval_post, k = 5,rank_col_name = 'rank_position', id_cols = ['query','daypart'])
df_metric_dct.keys()

In [0]:
# for max=25
df_metric_dct = evaluator._compute_metrics(df = df_eval_post, k = 5,rank_col_name = 'rank_position', id_cols = ['query','daypart'])
df_metric_dct.keys()

#### deep dive

In [0]:
import pandas as pd

data_tag = "max50_filtered"

df_eval = pd.read_csv(f'/dbfs/songxu/llm/structured_query_eval_res_{data_tag}.csv', escapechar='\\')
df_eval_post = evaluator._postprocess_data(df_eval, df_orig)
print(df_eval_post.shape)
df_eval_post.head(2)

df_eval_post_max50 = df_eval_post.copy(deep=True)
df_eval_post_max100 = df_eval_post.copy(deep=True)

In [0]:
# for max=50
df_metric_dct = evaluator._compute_metrics(df = df_eval_post, k = 5,rank_col_name = 'rank_position', id_cols = ['query','daypart'])
df_metric_dct.keys()

In [0]:
import copy

df_metric_dct_max50 = copy.deepcopy(df_metric_dct)
df_metric_dct_max100 = copy.deepcopy(df_metric_dct)

In [0]:
ndcg50 = df_metric_dct_max50['ndcg_query_level_df']
ndcg100 = df_metric_dct_max100['ndcg_query_level_df']

In [0]:
ndcg_joined = ndcg50.join(ndcg100, lsuffix='_max50', rsuffix='_max100')
ndcg_joined.head(2)

In [0]:
ndcg_joined['score_diff'] = ndcg_joined['weighted_score_max50'] - ndcg_joined['weighted_score_max100']
ndcg_joined = ndcg_joined.sort_values('score_diff', ascending=False)
ndcg_joined.head()

In [0]:
df_eval_post_max50[df_eval_post_max50['query']=='beef pho'].sort_values('rank_position', ascending=True)

In [0]:
df_eval_post_max100[df_eval_post_max100['query']=='beef pho'].sort_values('rank_position', ascending=True)

# run in one step (offline)

In [0]:
evaluator = StructuredQueryStoreEvaluator(
    user_prompt_column_names_mapping={
      "search_query": "query",
      "most_relevant_20_items_in_the_store":"most_relevant_20_items_in_the_store_core_category",
      "store_summary": "store_summary",
      "store_price_dollar_sign": "store_price_range_sign",
      "store_rating": "store_average_rating",
      "store_and_consumer_distance_miles": "store_consumer_distance_in_miles", 
      "store_eta_minute": "store_eta",
      "store_address":'store_address', 
    },
    structured_llm_call_output_class=BinaryQueryStoreResponse,
    model="gemini-2.5-pro",  # or any model routed via Portkey
    temperature=0.0,
    max_tokens=1024,
    timeout=60,
    prefix="gemini-2.5-pro",
    data_save_path = '/dbfs/mnt/doordash-datalake/test_tmp/ml/datasets/hs/vox/'
)

df_test = df.iloc[0:10] ## load data
df_pre_processed, df_eval_post, df_metrics_map = evaluator.run_pipeline(
  df = df_test,
  df_orig = df_test,
  run_data_preparation=True, 
  run_eval=True, 
  run_metrics=True,
  max_workers = 16,
  rank_col_name = 'rank_position',
  id_cols = ['query','daypart'], 
)

# only run post-processing
- if running evaluation through Metaflow, then for post-processing, directly run this cmd line

In [0]:
df_eval = pd.read_csv('/dbfs/mnt/doordash-datalake/test_tmp/ml/datasets/hs/vox/structured_query_eval_res.csv') ## load data
df_test = df_eval
df_orig = pd.read_csv('/dbfs/FileStore/heathersong/llm_as_judge/Structured_Query_Evaluation_0905___end_results.csv')
df_orig['weekday']= 'wed'
evaluator = StructuredQueryStoreEvaluator(
    user_prompt_column_names_mapping={
      "search_query": "query",
      "most_relevant_20_items_in_the_store":"most_relevant_20_items_in_the_store_core_category",
      "store_summary": "store_summary",
      "store_price_dollar_sign": "store_price_range_sign",
      "store_rating": "store_average_rating",
      "store_and_consumer_distance_miles": "store_consumer_distance_in_miles", 
      "store_eta_minute": "store_eta",
      "store_address":'store_address', 
    },
    structured_llm_call_output_class=BinaryQueryStoreResponse,
    model="gemini-2.5-pro",  # or any model routed via Portkey
    temperature=0.0,
    max_tokens=1024,
    timeout=60,
    prefix="gemini-2.5-pro",
    data_save_path = '/dbfs/mnt/doordash-datalake/test_tmp/ml/datasets/hs/vox/'
)
df_pre_processed, df_eval_post, df_metrics_map = evaluator.run_pipeline(
  df = df_test,
  df_orig = df_orig,
  run_data_preparation=False, ## change this to false 
  run_eval=False,  ## change this to false 
  run_metrics=True,
  max_workers = 16,
  rank_col_name = 'rank_position',
  id_cols = ['query','daypart'], 
)

# Note - running parameters in Metaflow


https://docs.google.com/document/d/1KeuhcbSoNL8Bg7CtAe6PHU3bkQxp7hCFXWN1yLZLeQk/edit?tab=t.0